<a href="https://colab.research.google.com/github/arbouria/Notas-Aprendizaje-y-Comportamiento-Adaptable-I/blob/main/Copia_de_Simulador_del_Modelo_Bayesiano_de_Punto_de_Cambio_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Novedades en esta Versión
Botón "Forzar Bloque Aleatorio":

He añadido un botón que, al presionarlo, añade un nuevo bloque de ensayos a la simulación.

La probabilidad de recompensa y la duración de este nuevo bloque se generan aleatoriamente, creando un entorno impredecible para que el modelo se adapte.

Aprendizaje de la Tasa de Peligro (H):

El modelo ya no asume una tasa de peligro fija. Ahora tiene una creencia interna sobre H y la actualiza en cada ensayo.

Se añade un nuevo parámetro y slider, Gamma (
gamma_E), que es la "tasa de aprendizaje para la tasa de peligro". Un valor alto hace que el modelo cambie su creencia sobre H muy rápidamente.

Se incluye un cuarto gráfico que muestra cómo la estimación de H del modelo (en azul) evoluciona con el tiempo, intentando converger a la frecuencia real de los cambios que ocurren.

Cómo Usarlo y Qué Observar
Ejecuta la celda. Verás una simulación inicial con dos bloques.

Presiona el botón "Forzar Bloque Aleatorio". La simulación se extenderá con un nuevo bloque de duración y probabilidad aleatorias, y las gráficas se volverán a dibujar. Hazlo varias veces para crear un entorno complejo.

Observa el Gráfico 4 (el nuevo):

La línea azul (Estimación de H del Modelo) es la creencia del modelo sobre la frecuencia de los cambios.

La línea naranja (H Real Promedio) es la frecuencia real de cambios en el entorno que has creado.

Verás cómo, después de cada punto de cambio (donde la CPP se dispara), la línea azul se ajusta. Con el tiempo, debería empezar a aproximarse a la línea naranja.

Juega con el slider de Gamma (
gamma_E):

Un valor bajo de
gamma_E hará que la línea azul se mueva muy lentamente. El modelo será "terco" en su creencia sobre la frecuencia de los cambios.

Un valor alto de
gamma_E hará que la línea azul reaccione muy bruscamente a la CPP más reciente, haciendo que la estimación de H sea mucho más volátil.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import FloatSlider, IntSlider, Button, VBox, HBox, Layout, Output
from IPython.display import display
import random

# --- Lógica Central del Simulador (Actualizada) ---

class ChangePointSimulator:
    """
    Simulador de un agente Bayesiano que aprende la tasa de peligro (H)
    en un entorno con puntos de cambio.
    """
    def __init__(self, initial_h=0.05, gamma_e=0.01):
        # Parámetros del modelo y del meta-aprendizaje
        self.gamma_e = gamma_e  # Tasa de aprendizaje para la Tasa de Peligro
        self.initial_h = initial_h
        self.reset()

    def reset(self):
        """Resetea el estado del simulador para una nueva ejecución."""
        self.V = 0.5            # Valor esperado inicial (creencia inicial)
        self.uncertainty = 1.0  # Incertidumbre inicial sobre el valor
        self.h_estimate = self.initial_h # Estimación de H se resetea al valor inicial

        # Historial para graficar
        self.history = {
            'rewards': [], 'predictions': [], 'learning_rates': [],
            'cpp': [], 'uncertainty': [], 'h_estimate': []
        }

    def _log_likelihood_bernoulli(self, outcome, prediction):
        """Calcula el logaritmo de la verosimilitud de un resultado Bernoulli."""
        pred = np.clip(prediction, 1e-6, 1 - 1e-6) # Evitar log(0)
        return outcome * np.log(pred) + (1 - outcome) * np.log(1 - pred)

    def update(self, reward):
        """
        Ejecuta un único ensayo de la simulación.
        """
        self.history['predictions'].append(self.V)
        self.history['rewards'].append(reward)
        self.history['uncertainty'].append(self.uncertainty)
        self.history['h_estimate'].append(self.h_estimate)

        # Usar la estimación actual de H para los cálculos
        log_likelihood_no_change = self._log_likelihood_bernoulli(reward, self.V)
        log_likelihood_change = self._log_likelihood_bernoulli(reward, 0.5)

        log_prior_change = np.log(self.h_estimate)
        log_prior_no_change = np.log(1 - self.h_estimate)

        # Calcular CPP con la H actual
        log_posterior_num = log_likelihood_change + log_prior_change
        log_posterior_den1 = log_likelihood_no_change + log_prior_no_change

        max_log = np.maximum(log_posterior_num, log_posterior_den1)
        log_denominator = max_log + np.log(np.exp(log_posterior_num - max_log) + np.exp(log_posterior_den1 - max_log))

        cpp = np.exp(log_posterior_num - log_denominator)
        self.history['cpp'].append(cpp)

        # Actualizar Tasa de Aprendizaje (alpha)
        alpha = cpp * 1.0 + (1 - cpp) * self.uncertainty
        self.history['learning_rates'].append(alpha)

        # Actualizar Valor (V)
        prediction_error = reward - self.V
        self.V = self.V + alpha * prediction_error

        # Actualizar Incertidumbre
        self.uncertainty = self.uncertainty * (1 - alpha) + 0.1 * cpp
        self.uncertainty = np.clip(self.uncertainty, 0.1, 1.0)

        # --- NUEVO: Aprender la tasa de peligro (H) ---
        # El modelo actualiza su creencia sobre H basándose en la CPP reciente.
        h_prediction_error = cpp - self.h_estimate
        self.h_estimate = self.h_estimate + self.gamma_e * h_prediction_error
        self.h_estimate = np.clip(self.h_estimate, 1e-4, 0.5) # Mantener H en un rango razonable

# --- Interfaz Interactiva (Reestructurada) ---

# Contenedor para las gráficas
plot_output = Output()

# Definir widgets de control
style = {'description_width': 'initial'}
layout = Layout(width='350px')
gamma_e_slider = FloatSlider(value=0.02, min=0.001, max=0.1, step=0.005, description='Gamma (γE - Learn Rate de H):', style=style, layout=layout, readout_format='.3f')
initial_h_slider = FloatSlider(value=0.05, min=0.01, max=0.2, step=0.01, description='H Inicial del Modelo:', style=style, layout=layout)
reset_button = Button(description="Reiniciar Simulación", button_style='danger')
add_block_button = Button(description="Forzar Bloque Aleatorio", button_style='success')

# Estado de la simulación (lista de bloques)
simulation_blocks = [
    {'prob': 0.8, 'len': 75},
    {'prob': 0.2, 'len': 75}
]

def run_and_plot(blocks):
    """Función central que corre la simulación y actualiza las gráficas."""
    with plot_output:
        plot_output.clear_output(wait=True)

        reward_probabilities = np.concatenate([np.full(b['len'], b['prob']) for b in blocks])
        rewards = np.random.binomial(1, reward_probabilities)

        simulator = ChangePointSimulator(initial_h=initial_h_slider.value, gamma_e=gamma_e_slider.value)
        for reward in rewards:
            simulator.update(reward)

        fig, axes = plt.subplots(4, 1, figsize=(14, 12), sharex=True)
        fig.suptitle('Simulador de Modelo Bayesiano con Aprendizaje de Tasa de Peligro', fontsize=16, y=0.97)
        trials = np.arange(1, len(rewards) + 1)

        # Puntos de cambio reales para las líneas verticales
        change_points = np.cumsum([b['len'] for b in blocks[:-1]])

        # Gráfico 1: Recompensas y Predicciones
        ax1 = axes[0]
        ax1.plot(trials, simulator.history['predictions'], 'g-', label='Predicción del Modelo ($V_t$)')
        ax1.scatter(trials, simulator.history['rewards'], c='k', marker='|', alpha=0.7, label='Recompensa Observada ($R_t$)')
        ax1.plot(trials, reward_probabilities, 'r--', alpha=0.6, label='Probabilidad Real')
        for cp in change_points: ax1.axvline(cp + 0.5, color='grey', linestyle=':', lw=2)
        ax1.set_ylabel('Probabilidad'), ax1.set_title('1. Recompensas y Valor Predicho'), ax1.legend(loc='best'), ax1.grid(True, alpha=0.3)

        # Gráfico 2: Tasa de Aprendizaje
        ax2 = axes[1]
        ax2.plot(trials, simulator.history['learning_rates'], 'b-', label='Tasa de Aprendizaje ($\\alpha_t$)')
        for cp in change_points: ax2.axvline(cp + 0.5, color='grey', linestyle=':', lw=2)
        ax2.set_ylabel('$\\alpha$'), ax2.set_title('2. Tasa de Aprendizaje Dinámica'), ax2.set_ylim(0, 1.1), ax2.grid(True, alpha=0.3)

        # Gráfico 3: CPP
        ax3 = axes[2]
        ax3.plot(trials, simulator.history['cpp'], 'm-', label='Probabilidad de Cambio ($CPP_t$)')
        for cp in change_points: ax3.axvline(cp + 0.5, color='grey', linestyle=':', lw=2)
        ax3.set_ylabel('Probabilidad'), ax3.set_title('3. Creencia sobre Punto de Cambio (CPP)'), ax3.set_ylim(0, 1.1), ax3.grid(True, alpha=0.3)

        # Gráfico 4: Aprendizaje de la Tasa de Peligro (H)
        ax4 = axes[3]
        ax4.plot(trials, simulator.history['h_estimate'], 'c-', label='Estimación de H del Modelo')
        # Tasa de peligro real (promedio)
        avg_h = 1 / np.mean([b['len'] for b in blocks])
        ax4.axhline(avg_h, color='orange', linestyle='--', label=f'H Real Promedio ({avg_h:.3f})')
        for cp in change_points: ax4.axvline(cp + 0.5, color='grey', linestyle=':', lw=2)
        ax4.set_xlabel('Ensayo'), ax4.set_ylabel('Tasa de Peligro (H)'), ax4.set_title('4. Aprendizaje de la Tasa de Peligro'), ax4.legend(loc='best'), ax4.grid(True, alpha=0.3)

        plt.tight_layout(rect=[0, 0, 1, 0.96])
        plt.show()

# --- Funciones de los botones ---
def on_reset_button_clicked(b):
    global simulation_blocks
    simulation_blocks = [{'prob': 0.8, 'len': 75}, {'prob': 0.2, 'len': 75}] # Resetea a la config inicial
    run_and_plot(simulation_blocks)

def on_add_block_button_clicked(b):
    global simulation_blocks
    # Añade un nuevo bloque con parámetros aleatorios
    new_prob = round(random.uniform(0.1, 0.9), 2)
    new_len = random.randint(40, 120)
    simulation_blocks.append({'prob': new_prob, 'len': new_len})
    run_and_plot(simulation_blocks)

def on_slider_change(change):
    run_and_plot(simulation_blocks)

# Conectar widgets a sus funciones
reset_button.on_click(on_reset_button_clicked)
add_block_button.on_click(on_add_block_button_clicked)
gamma_e_slider.observe(on_slider_change, names='value')
initial_h_slider.observe(on_slider_change, names='value')

# Organizar la UI y mostrarla
controls = VBox([
    HBox([initial_h_slider, gamma_e_slider]),
    HBox([reset_button, add_block_button])
])
display(controls, plot_output)

# Ejecución inicial
run_and_plot(simulation_blocks)

Output()